In [81]:
%logstop
%logstart -rtq ~/.logs/nlp.py append
import seaborn as sns
sns.set()

In [7]:
from static_grader import grader

# Collins Natural Language Processing (NLP) project

## Introduction

The objective of this project is to gain experience with natural language processing and how to use text data to train a machine learning model to make predictions. For the miniproject, we will be working with product review text from Amazon. The reviews are for only products in the "Electronics" category. The objective is to train a model to predict the rating, ranging from 1 to 5 stars.

## Scoring

For most of the questions, you will be asked to submit the `predict` method of your trained model to the grader. The grader will use the passed `predict` method to evaluate how your model performs on a test set with respect to a reference model. The grader uses the [R<sup>2</sup>-score](https://scikit-learn.org/stable/modules/model_evaluation.html#r2-score) for model evaluation. If your model performs better than the reference solution, then you can score higher than 1.0. For the last question, you will submit the results of an analysis and your passed answer will be compared directly to the reference solution.

## Downloading and loading the data

The data set is available on Amazon S3 and comes as a compressed file where each line is a JSON object. To load the data set, we will need to use the `gzip` library to open the file and decode each JSON into a Python dictionary. In the end, we have a list of dictionaries, where each dictionary represents an observation.

In [8]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_electronics_reviews_training.json.gz -nc -P ./data

mkdir: cannot create directory ‘data’: File exists
File ‘./data/amazon_electronics_reviews_training.json.gz’ already there; not retrieving.



In [9]:
import gzip
import simplejson as json

with gzip.open("data/amazon_electronics_reviews_training.json.gz", "r") as f:                                  
    data = [json.loads(line) for line in f]

In [10]:
data[:2]

[{'reviewerID': 'A238V1XTSK9NFE',
  'asin': 'B00004VX3T',
  'reviewerName': 'Andrew Lynn',
  'helpful': [2, 2],
  'reviewText': "I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automa

The ratings are stored in the keyword `"overall"`. You should create an array of the ratings for each review, preferably using list comprehensions.

In [11]:
ratings = [x['overall'] for x in data]

In [13]:
ratings[:5]

[5.0, 1.0, 4.0, 5.0, 3.0]

**Note**, the test set used by the grader is in the same format as that of `data`, a list of dictionaries. Your trained model needs to accept data in the same format. Thus, you should use `Pipeline` when constructing your model so that all necessary transformation needed are encapsulated into a single estimator object.

## Question 1: Bag of words model

Construct a machine learning model trained on word counts using the bag of words algorithm. Remember, the bag of words is implemented with `CountVectorizer`. Some things you should consider:

* The reference solution uses a linear model and you should as well; use either `Ridge` or `SGDRegressor`.
* The text review is stored in the key `"reviewText"`. You will need to construct a custom transformer to extract out the value of this key. It will be the first step in your pipeline.
* Consider what hyperparameters you will need to tune for your model.
* Subsampling the training data will boost training times, which will be helpful when determining the best hyperparameters to use. Note, your final model will perform best if it is trained on the full data set.
* Including stop words may help with performance.

In [19]:
from sklearn.base import BaseEstimator, TransformerMixin
class KeySelector(BaseEstimator, TransformerMixin):
    
    def __init__(self, key):
        self.key = key
        
    def fit(self,X, y=None):
        return self
    
    def transform(self,X):
        return [d[self.key]for d in X]
        
    
    

In [22]:
KeySelector("reviewText").fit_transform(data)[:3]

["I bought this mouse to use with my laptop because I don't like those little touchpads.  I could not be happier.Since it's USB, I can plug it in with the computer already on and expect it to work automatically.  Since it's optical (the new kind, not to be confused with the old Sun optical mice that required a special checkered mouse pad) it works on most surfaces, including my pant legs, my couch, and random tables that I put my laptop down on.  It's also light and durable, features that help with portability.The wheel is surprisingly useful.  In addition to scrolling, it controls zoom and pan in programs like Autocad and 3D Studio Max.  I can no longer bear using either of these programs without it.One complaint - the software included with the Internet navigation features is useless.  Don't bother installing it if you have a newer Windows version that automatically supports wheel mice.  Just plug it in and use it - it's that easy.",
 'One by one, all of the discs went bad within a 6

In [17]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
import numpy as np

In [35]:

bag_of_words_model = Pipeline([
    ("select_text", KeySelector('reviewText')),
    ("vectorizer", CountVectorizer(stop_words = 'english')),
    ('regressor', GridSearchCV(Ridge(), 
                              param_grid= {'alpha': np.logspace(-1,3,10)},
                              cv=5))
                           
])

In [36]:
bag_of_words_model.fit(data, ratings)

Pipeline(steps=[('select_text', KeySelector(key='reviewText')),
                ('vectorizer', CountVectorizer(stop_words='english')),
                ('regressor',
                 GridSearchCV(cv=5, estimator=Ridge(),
                              param_grid={'alpha': array([1.00000000e-01, 2.78255940e-01, 7.74263683e-01, 2.15443469e+00,
       5.99484250e+00, 1.66810054e+01, 4.64158883e+01, 1.29154967e+02,
       3.59381366e+02, 1.00000000e+03])}))])

In [39]:
bag_of_words_model.score(data, ratings)

0.40109149298944113

In [40]:
bag_of_words_model.predict(data)[:10]

array([4.07881023, 3.34877818, 4.71352638, 4.44751994, 3.93731306,
       4.8432953 , 4.46580972, 4.11563883, 3.96989081, 4.08209632])

In [41]:
grader.score.nlp__bag_of_words_model(bag_of_words_model.predict)

Your score: 1.045


In [43]:
bag_of_words_model.named_steps['regressor'].best_params_

{'alpha': 359.38136638046257}

In [44]:
bag_of_words_model.named_steps['vectorizer'].vocabulary_

{'bought': 11721,
 'mouse': 38703,
 'use': 61859,
 'laptop': 34336,
 'don': 20569,
 'like': 35145,
 'little': 35373,
 'touchpads': 59469,
 'happier': 28673,
 'usb': 61826,
 'plug': 44490,
 'computer': 15876,
 'expect': 23718,
 'work': 64727,
 'automatically': 9354,
 'optical': 41572,
 'new': 39921,
 'kind': 33773,
 'confused': 16132,
 'old': 41202,
 'sun': 56314,
 'mice': 37560,
 'required': 49121,
 'special': 54373,
 'checkered': 14216,
 'pad': 42507,
 'works': 64783,
 'surfaces': 56566,
 'including': 31193,
 'pant': 42712,
 'legs': 34668,
 'couch': 16993,
 'random': 47404,
 'tables': 57199,
 'light': 35056,
 'durable': 21246,
 'features': 24656,
 'help': 29346,
 'portability': 44905,
 'wheel': 63961,
 'surprisingly': 56621,
 'useful': 61885,
 'addition': 6425,
 'scrolling': 51372,
 'controls': 16607,
 'zoom': 65799,
 'pan': 42643,
 'programs': 46041,
 'autocad': 9312,
 '3d': 3023,
 'studio': 55894,
 'max': 36889,
 'longer': 35615,
 'bear': 10426,
 'using': 61944,
 'complaint': 15716,

In [54]:
bag_of_words_model.named_steps['vectorizer'].get_feature_names()[11721]

'bought'

## Question 2: Normalized model

Using raw counts will not be as effective compared if we had normalized the counts. There are several ways to normalize raw counts; the `HashingVectorizer` class has the keyword `norm` and there is also the `TfidfTransformer` and `TfidfVectorizer` that perform tf-idf weighting on the counts. Apply normalization to your model to improve performance.

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

normalized_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer', TfidfVectorizer(stop_words='english')),
    ('model', Ridge())])
normalized_model.fit(data, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer', TfidfVectorizer(stop_words='english')),
                ('model', Ridge())])

In [61]:
normalized_model.score(data,ratings)

0.6025417722330686

In [62]:
grader.score.nlp__normalized_model(normalized_model.predict)

Your score: 0.993


## Question 3: Bigrams model

The model performance may increase when including additional features generated by counting bigrams. Include bigrams to your model. When using more features, the risk of overfitting increases. Make sure you try to minimize overfitting as much as possible.

In [37]:
bigrams_model = Pipeline([
    ('selector', KeySelector('reviewText')),
    ('vectorizer',TfidfVectorizer(stop_words = 'english', ngram_range=(1,2))),
    ('regressor', Ridge())
    
]) 

In [38]:
bigrams_model.fit(data, ratings)

Pipeline(steps=[('selector', KeySelector(key='reviewText')),
                ('vectorizer',
                 TfidfVectorizer(ngram_range=(1, 2), stop_words='english')),
                ('regressor', Ridge())])

In [39]:
grader.score.nlp__bigrams_model(bigrams_model.predict)

Your score: 1.033


In [46]:
bigrams_model.score(data, ratings)

0.8263560931212581

In [45]:
bigrams_model.named_steps['vectorizer'].vocabulary_

{'bought': 179469,
 'mouse': 869568,
 'use': 1446249,
 'laptop': 735441,
 'don': 415493,
 'like': 764355,
 'little': 779051,
 'touchpads': 1396630,
 'happier': 609065,
 'usb': 1445198,
 'plug': 1008698,
 'computer': 301445,
 'expect': 478258,
 'work': 1525146,
 'automatically': 123386,
 'optical': 933215,
 'new': 894914,
 'kind': 723057,
 'confused': 308964,
 'old': 923854,
 'sun': 1330230,
 'mice': 842313,
 'required': 1139500,
 'special': 1282630,
 'checkered': 259028,
 'pad': 952030,
 'works': 1530770,
 'surfaces': 1338780,
 'including': 670869,
 'pant': 959802,
 'legs': 747698,
 'couch': 334705,
 'random': 1092274,
 'tables': 1348535,
 'light': 760644,
 'durable': 431713,
 'features': 508246,
 'help': 629674,
 'portability': 1021144,
 'wheel': 1508816,
 'surprisingly': 1340360,
 'useful': 1454740,
 'addition': 70511,
 'scrolling': 1191601,
 'controls': 323964,
 'zoom': 1550557,
 'pan': 958370,
 'programs': 1059677,
 'autocad': 122728,
 '3d': 31538,
 'studio': 1320024,
 'max': 82795

## Question 4: Polarity analysis

Let's derive some insight from our analysis. We want to determine the most polarizing words in the corpus of reviews. In other words, we want identify words that strongly signal a review is either positive or negative. For example, we understand a word like "terrible" will mostly appear in negative rather than positive reviews. The naive Bayes model calculates probabilities such as $P(\text{terrible } | \text{ negative})$, the probability the word "terrible" appears in the text, given that the review is negative. Using these probabilities, we can derive a **polarity score** for each counted word,

$$
\text{polarity} =  \log\left(\frac{P(\text{word } | \text{ positive})}{P(\text{word } | \text{ negative})}\right).
$$ 

The polarity analysis is an example where a simpler model offers more explicability than a more complicated model. For this question, you are asked to determine the top twenty-five words with the largest positive **and** largest negative polarity, for a total of fifty words. For this analysis, you should:

1. Use the naive Bayes model, `MultinomialNB`.
1. Use tf-idf weighting.
1. Remove stop words.

A trained naive Bayes model stores the log of the probabilities in the attribute `feature_log_prob_`. It is a NumPy array of shape (number of classes, the number of features). You will need the mapping between feature index to word. For this problem, you will use a different data set; it has been processed to only include reviews with one and five stars. You can download it below.

In [3]:
%%bash
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz -nc -P ./data

--2021-08-13 11:26:21--  http://dataincubator-wqu.s3.amazonaws.com/mldata/amazon_one_and_five_star_reviews.json.gz
Resolving dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)... 52.217.43.76
Connecting to dataincubator-wqu.s3.amazonaws.com (dataincubator-wqu.s3.amazonaws.com)|52.217.43.76|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2970853 (2.8M) [application/x-gzip]
Saving to: ‘./data/amazon_one_and_five_star_reviews.json.gz’

     0K .......... .......... .......... .......... ..........  1% 3.24M 1s
    50K .......... .......... .......... .......... ..........  3% 6.63M 1s
   100K .......... .......... .......... .......... ..........  5% 6.98M 1s
   150K .......... .......... .......... .......... ..........  6% 46.9M 0s
   200K .......... .......... .......... .......... ..........  8% 62.2M 0s
   250K .......... .......... .......... .......... .......... 10% 47.5M 0s
   300K .......... .......... .......... .......... .........

In order to avoid memory issues, let's delete the older data.

In [ ]:
del data, ratings

In [24]:
import numpy as np
import gzip
import simplejson as json

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer

with gzip.open("data/amazon_one_and_five_star_reviews.json.gz", "r") as f:
    data_polarity = [json.loads(line) for line in f]

ratings = [x['overall'] for x in data_polarity]

In [25]:
data_polarity[:3]

[{'reviewerID': 'A31RSJTGLVV3TR',
  'asin': 'B000093UDP',
  'reviewerName': 'T. Wayne',
  'helpful': [11, 12],
  'reviewText': "This worked perfectly for about 8 rewinds.  But once it eats one of your precious tapes you'll probably be as dissapointed as I was.  Buyer beware... these rewinders are mostly the same look; but marketed by different companies.  They all eat tape.",
  'overall': 1.0,
  'summary': "It's a super tape-eater",
  'unixReviewTime': 1210550400,
  'reviewTime': '05 12, 2008'},
 {'reviewerID': 'A2Y739CRM15WDL',
  'asin': 'B00005MNSR',
  'reviewerName': 'Sires "I like mysteries (particularly British...',
  'helpful': [0, 0],
  'reviewText': "This was a choice in my gold box or I might not have bought it. I would have been missing out on a very good monitor if I hadn't.I just bought a new 2.53 gigahertz multi media computer running XP home edition and wanted a monitor with a small footprint but still able to handle graphics intense functions.  Set up was a breeze.  I ju

In [34]:
pipe = Pipeline([
    ("seector", KeySelector('reviewText')),
    ('vectorizer',TfidfVectorizer(stop_words = 'english')),
    ('classifier', MultinomialNB())
])
pipe.fit(data_polarity, ratings)

Pipeline(steps=[('seector', KeySelector(key='reviewText')),
                ('vectorizer', TfidfVectorizer(stop_words='english')),
                ('classifier', MultinomialNB())])

In [30]:
pipe.predict(data_polarity)[:10]

array([1., 5., 1., 5., 5., 5., 1., 5., 5., 5.])

In [36]:
nb = pipe.named_steps['classifier'] 

In [37]:
nb.classes_

array([1., 5.])

In [42]:
nb.feature_log_prob_.shape

(2, 25411)

In [56]:
words = pipe.named_steps['vectorizer'].get_feature_names()

In [59]:
words[10000]

'frequent'

In [60]:
log_prob = nb.feature_log_prob_

In [67]:
polarity = log_prob[0] - log_prob[1]

In [68]:
polarity

array([ 0.70107467, -0.32918854, -0.29068542, ...,  0.61837882,
        0.10651328,  0.19406434])

In [75]:
indices = np.argsort(polarity)

In [76]:
indices_top = np.hstack([indices[:25], indices[-25:]])

In [79]:
indices_top

array([11275,  3609, 17631, 16544, 14775, 17108,  2511, 21599, 13906,
       11965,  8872,  4034, 16902,   524,  5008, 10935,  3302, 17109,
        7955,  6371, 16703, 22319,  4351,  9259,  8191, 18496, 22622,
        7410, 21414, 18037, 23695, 11446,  3303, 23403, 17067,  3775,
        6871, 23059, 25037, 25043, 23889, 10263, 19011, 22395, 12793,
       25041, 19012, 19009, 24520, 18484])

In [80]:
len(indices_top)

50

In [82]:
[words[i] for i in indices_top]

['highly',
 'beat',
 'protects',
 'perfect',
 'monopod',
 'portrait',
 'amazing',
 'sturdy',
 'macro',
 'incredible',
 'excellent',
 'bokeh',
 'pleased',
 '200mm',
 'charm',
 'handy',
 'awesome',
 'portraits',
 'dslr',
 'crisp',
 'photography',
 'telephoto',
 'buck',
 'fantastic',
 'easy',
 'refused',
 'threw',
 'disappointing',
 'stopped',
 'randomly',
 'unreliable',
 'horrible',
 'awful',
 'unacceptable',
 'poor',
 'beware',
 'defective',
 'trash',
 'worse',
 'worthless',
 'useless',
 'garbage',
 'returned',
 'terrible',
 'junk',
 'worst',
 'returning',
 'return',
 'waste',
 'refund']

In [83]:
top_50 = [words[i] for i in indices_top]

In [84]:
grader.score.nlp__most_polar(top_50)

Your score: 1.000


*Copyright &copy; 2021 WorldQuant University. This content is licensed solely for personal use. Redistribution or publication of this material is strictly prohibited.*